# Regressão III - Preparação da base de dados

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
# carrega o arquivo bruto
arquivo = 'previsao_de_renda_raw.csv'
df = pd.read_csv(arquivo)
df.head()

,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,5762,M,S,N,0,Empresário,Superior completo,Casado,Casa,47,16.717808,2.0,11138.14
1,2015-01-01,1279,M,S,N,0,Assalariado,Superior incompleto,Casado,Casa,30,9.600000,2.0,2424.81
2,2015-01-01,14111,M,S,N,0,Assalariado,Superior completo,Casado,Casa,28,8.208219,2.0,13749.66
3,2015-01-01,8189,M,N,S,2,Assalariado,Médio,Casado,Casa,44,1.301370,4.0,2361.84
4,2015-01-01,1830,F,N,S,2,Assalariado,Superior completo,Casado,Casa,33,1.254795,4.0,790.78


In [21]:
# tamanho da base
df.shape

(750000, 14)

#### Tipos de variáveis

Checando os tipos de variáveis

In [22]:
df.dtypes

data_ref                  object
index                      int64
sexo                      object
posse_de_veiculo          object
posse_de_imovel           object
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                      int64
tempo_emprego            float64
qt_pessoas_residencia    float64
renda                    float64
dtype: object

In [23]:
# tipo da variavel data_ref antes da transformacao
print(type(df.data_ref[0]))

<class 'str'>


In [24]:
# transforma a coluna data_ref para o tipo datetime
df.data_ref = pd.to_datetime(df.data_ref, format='%Y-%m-%d')

# tipo da variavel data_ref pos transformacao
df.data_ref.head(5)

0   2015-01-01
1   2015-01-01
2   2015-01-01
3   2015-01-01
4   2015-01-01
Name: data_ref, dtype: datetime64[ns]

Convertendo os valores 'S' e 'N' para valores booleanos nas colunas "posse_veiculo" e "posse_imovel"

In [25]:
df[["posse_de_veiculo", "posse_de_imovel"]] = df[["posse_de_veiculo", "posse_de_imovel"]].replace({'S': True, 'N': False}).astype(bool)

#### Valores nulos e faltantes

In [26]:
df.isna().any().to_frame()

,0
data_ref,False
index,False
sexo,False
posse_de_veiculo,False
posse_de_imovel,False
qtd_filhos,False
tipo_renda,False
educacao,False
estado_civil,False
tipo_residencia,False


In [27]:
df.isnull().any().to_frame()

,0
data_ref,False
index,False
sexo,False
posse_de_veiculo,False
posse_de_imovel,False
qtd_filhos,False
tipo_renda,False
educacao,False
estado_civil,False
tipo_residencia,False


A coluna tempo_emprego possui valores nulos e faltantes.
Vamos operar sobre esses valores.

In [28]:
# nulos
df.tempo_emprego.isnull().value_counts().to_frame()

,tempo_emprego
False,624240
True,125760


In [29]:
# faltantes
df.tempo_emprego.isna().value_counts().to_frame()

,tempo_emprego
False,624240
True,125760


Como a quantidade de valores faltantes e nulos são iguais, vale a pena conferir se são os mesmos indíces, ou valores diferentes.

In [30]:
checagem_index = df.tempo_emprego.isna().index.equals(df.tempo_emprego.isnull().index)
print("Os indices são iguais." if checagem_index else "Os indices são diferentes.")


Os indices são iguais.


Sendo os indices iguais, e como o número de entradas é grande, não queremos perder essa quantidade de dados. Portanto, os valores vazios/nulos serão preenchidos com a média.

In [31]:
# preenchendo com a média
df.tempo_emprego.fillna(df.tempo_emprego.mean(), inplace=True)

In [32]:
# checando se funcionou
df.tempo_emprego.isna().value_counts().to_frame()

,tempo_emprego
False,750000


#### Separando as bases de treino e teste

Os últimos três meses serão destinados a treino.

In [33]:
df.data_ref.min()

Timestamp('2015-01-01 00:00:00')

In [34]:
df.data_ref.max()

Timestamp('2016-03-01 00:00:00')

In [44]:
data_inicio = ('2016-01-01')
data_final = ('2016-03-01')
df_teste = df[(df['data_ref'] >= data_inicio) & (df['data_ref'] <= data_final)]
df_teste

,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
600000,2016-01-01,12946,F,False,True,0,Pensionista,Médio,Casado,Casa,61,7.746165,2.0,2137.61
600001,2016-01-01,8872,F,False,True,2,Servidor público,Superior completo,Casado,Casa,27,9.301370,4.0,13884.35
600002,2016-01-01,16279,F,True,True,3,Assalariado,Médio,Casado,Casa,38,10.967123,5.0,11042.47
600003,2016-01-01,8158,M,True,True,3,Servidor público,Médio,Casado,Casa,35,5.553425,5.0,1333.82
600004,2016-01-01,9356,F,True,False,0,Assalariado,Médio,Viúvo,Casa,63,9.265753,1.0,17633.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,2016-03-01,11611,F,False,True,1,Assalariado,Superior completo,Casado,Casa,29,8.665753,3.0,2930.40
749996,2016-03-01,9144,F,False,True,0,Pensionista,Superior completo,Casado,Casa,65,7.746165,2.0,4084.37
749997,2016-03-01,16583,F,False,True,0,Assalariado,Superior completo,Casado,Casa,33,10.969863,2.0,4339.66
749998,2016-03-01,3024,F,True,True,1,Empresário,Superior completo,União,Casa,28,8.219178,3.0,9159.49


In [45]:
df_treino = df.drop(df_teste.index)
df_treino

,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,5762,M,True,False,0,Empresário,Superior completo,Casado,Casa,47,16.717808,2.0,11138.14
1,2015-01-01,1279,M,True,False,0,Assalariado,Superior incompleto,Casado,Casa,30,9.600000,2.0,2424.81
2,2015-01-01,14111,M,True,False,0,Assalariado,Superior completo,Casado,Casa,28,8.208219,2.0,13749.66
3,2015-01-01,8189,M,False,True,2,Assalariado,Médio,Casado,Casa,44,1.301370,4.0,2361.84
4,2015-01-01,1830,F,False,True,2,Assalariado,Superior completo,Casado,Casa,33,1.254795,4.0,790.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,2015-12-01,4650,F,False,True,1,Assalariado,Superior incompleto,Solteiro,Casa,37,4.441096,2.0,1893.35
599996,2015-12-01,9717,M,True,True,0,Empresário,Médio,Casado,Estúdio,44,2.345205,2.0,3080.48
599997,2015-12-01,7485,M,True,True,0,Assalariado,Médio,Casado,Casa,28,0.736986,2.0,5131.20
599998,2015-12-01,15500,F,True,True,1,Empresário,Superior completo,Solteiro,Casa,27,6.791781,2.0,20449.25


In [49]:
# reset do index do testes
df_teste.reset_index(drop=True, inplace=True)
df_teste

,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2016-01-01,12946,F,False,True,0,Pensionista,Médio,Casado,Casa,61,7.746165,2.0,2137.61
1,2016-01-01,8872,F,False,True,2,Servidor público,Superior completo,Casado,Casa,27,9.301370,4.0,13884.35
2,2016-01-01,16279,F,True,True,3,Assalariado,Médio,Casado,Casa,38,10.967123,5.0,11042.47
3,2016-01-01,8158,M,True,True,3,Servidor público,Médio,Casado,Casa,35,5.553425,5.0,1333.82
4,2016-01-01,9356,F,True,False,0,Assalariado,Médio,Viúvo,Casa,63,9.265753,1.0,17633.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,2016-03-01,11611,F,False,True,1,Assalariado,Superior completo,Casado,Casa,29,8.665753,3.0,2930.40
149996,2016-03-01,9144,F,False,True,0,Pensionista,Superior completo,Casado,Casa,65,7.746165,2.0,4084.37
149997,2016-03-01,16583,F,False,True,0,Assalariado,Superior completo,Casado,Casa,33,10.969863,2.0,4339.66
149998,2016-03-01,3024,F,True,True,1,Empresário,Superior completo,União,Casa,28,8.219178,3.0,9159.49


#### Salvando a base pronta para uso

Salvando para um arquivo feather para melhor perfomance de leitura/escrita

In [50]:
df_teste.to_feather('previsao_de_renda_teste.feather')
df_treino.to_feather('previsao_de_renda_treino.feather')